# SAC algorithm implementation

> In this notebook, we implement a state-of-the-art Multi Agent Reinforcement Leaning (MARL) algorithms **[SAC](https://arxiv.org/pdf/1801.01290)** in our environment. SAC is an off-policy actor-critic deep RL algorithm based on the maximum entropy RL framework. 


> Tutorial based on [SAC TorchRL Tutorial](https://github.com/pytorch/rl/blob/main/sota-implementations/multiagent/sac.py).

### Simulation overview

> We simulate our environment with an initial population of **20 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **10 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ the QMIX reinforcement learning algorithms to further refine their learning.

#### Imported libraries

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import os
import time
import torch

from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import nn
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torch.distributions import Categorical, OneHotCategorical
from torchrl._utils import logger as torchrl_logger
from torchrl.collectors import SyncDataCollector
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.envs import RewardSum, TransformedEnv
from torchrl.envs.libs.vmas import VmasEnv
from torchrl.envs.utils import ExplorationType, set_exploration_type
from torchrl.modules import ProbabilisticActor, TanhNormal, ValueOperator
from torchrl.modules.models.multiagent import MultiAgentMLP
from torchrl.objectives import DiscreteSACLoss, SACLoss, SoftUpdate, ValueEstimators

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from RouteRL.keychain import Keychain as kc
from RouteRL.environment.environment import TrafficEnvironment
from RouteRL.services.plotter import Plotter
from RouteRL.utilities import get_params

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#### Hyperparameters setting

In [2]:
params = get_params("params.json")

In [3]:
# Devices
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

machine_agents = params["agent_generation_parameters"]["new_machines_after_mutation"]


print("device is: ", device)
vmas_device = device  # The device where the simulator is run

# Sampling
frames_per_batch = 4 * machine_agents  # Number of team frames collected per training iteration
n_iters = 10  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 10  # Number of optimization steps per training iteration
minibatch_size = 2  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients
memory_size =  8 * machine_agents  # Size of the replay buffer
tau =  0.005
gamma = 0.99  # discount factor

device is:  cpu


#### Environment initialization

> In this example, the environment initially contains only human agents.

In [4]:
env = TrafficEnvironment(params[kc.RUNNER], params[kc.ENVIRONMENT], params[kc.SIMULATOR], params[kc.AGENT_GEN], params[kc.AGENTS], params[kc.PLOTTER])

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: C:\Program Files (x86)\Eclipse\Sumo\tools


In [5]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  20 

Agents are:  [Human 0, Human 1, Human 2, Human 3, Human 4, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19] 

Number of human agents is:  20 

Number of machine agents (autonomous vehicles) is:  0 



> Reset the environment and the connection with SUMO

In [6]:
env.start()
env.reset()

({}, {})

#### Human learning

In [7]:
num_episodes = 100

for episode in range(num_episodes):
    env.step()

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles). You can adjust the number of agents to be mutated in the <code style="color:white">/params.json</code> file.

In [8]:
env.mutation()

In [9]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  20 

Agents are:  [Machine 14, Machine 5, Machine 8, Machine 7, Machine 11, Machine 13, Machine 16, Machine 17, Machine 15, Machine 19, Human 0, Human 1, Human 2, Human 3, Human 4, Human 6, Human 9, Human 10, Human 12, Human 18] 

Number of human agents is:  10 

Number of machine agents (autonomous vehicles) is:  10 



> Create a group that contains all the machine (RL) agents.

>  **Hint:** As a feature of TorchRL multiagent, we are able to control the grouping of agents. We can group agents together (stacking their tensors) to leverage vectorization when passing them through the same neural network. We can split agents in different groups where they are heterogenous or should be processed by different neural netowkrs.

In [10]:
machine_list = []
for machines in env.machine_agents:
    machine_list.append(str(machines.id))
      
group = {'agents': machine_list}

#### PettingZoo environment wrapper

In [11]:
env = PettingZooWrapper(
    env=env,
    use_mask=True, # Whether to use the mask in the outputs. It is important for AEC environments to mask out non-acting agents.
    categorical_actions=True,
    done_on_any = False, # Whether the environment’s done keys are set by aggregating the agent keys using any() (when True) or all() (when False).
    group_map=group,
    device=device
)

> The environment is defined by a series of metadata that describe what can be expected during its execution. 

There are four specs to look at:

- <code style="color:white">action_spec</code> defines the action space;

- <code style="color:white">reward_spec</code> defines the reward domain;

- <code style="color:white">done_spec</code> defines the done domain;

- <code style="color:white">observation_spec</code> which defines the domain of all other outputs from environment steps;

In [12]:
print("action_spec:", env.full_action_spec, "\n\n")
print("reward_spec:", env.full_reward_spec, "\n\n")
print("done_spec:", env.full_done_spec, "\n\n")
print("observation_spec:", env.observation_spec, "\n\n")

action_spec: CompositeSpec(
    agents: CompositeSpec(
        action: DiscreteTensorSpec(
            shape=torch.Size([10]),
            space=DiscreteBox(n=3),
            device=cpu,
            dtype=torch.int64,
            domain=discrete), device=cpu, shape=torch.Size([10])), device=cpu, shape=torch.Size([])) 


reward_spec: CompositeSpec(
    agents: CompositeSpec(
        reward: UnboundedContinuousTensorSpec(
            shape=torch.Size([10, 1]),
            space=None,
            device=cpu,
            dtype=torch.float32,
            domain=continuous), device=cpu, shape=torch.Size([10])), device=cpu, shape=torch.Size([])) 


done_spec: CompositeSpec(
    done: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=cpu,
        dtype=torch.bool,
        domain=discrete),
    terminated: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=cpu,
        dtype=torch.bool,
        domain

> Agent group mapping

In [13]:
print("env.group is: ", env.group_map, "\n\n")

env.group is:  {'agents': ['14', '5', '8', '7', '11', '13', '16', '17', '15', '19']} 




#### Transforms

> We can append any TorchRL transform we need to our environment. These will modify its input/output in some desired way. In multi-agent contexts, it is paramount to provide explicitly the keys to modify.



Here we instatiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [14]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [15]:
reset_td = env.reset()

#### Policy network

> Instantiate an `MPL` that can be used in multi-agent contexts.

In [16]:
share_parameters_policy = False 

actor_net = torch.nn.Sequential(
    MultiAgentMLP(
        n_agent_inputs = env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs = env.action_spec.space.n,
        n_agents = env.n_agents,
        centralised=False,
        share_params=share_parameters_policy,
        device=device,
        depth=3,
        num_cells=64,
        activation_class=torch.nn.Tanh,
    ),
)

In [17]:
policy_module = TensorDictModule(
    actor_net,
    in_keys=[("agents", "observation")],
    out_keys=[("agents", "logits")],
) 

In [18]:
policy = ProbabilisticActor(
    module=policy_module,
    spec=env.action_spec,
    in_keys=[("agents", "logits")],
    out_keys=[env.action_key],
    distribution_class=Categorical,
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)

#### Critic network

> The critic reads the observations and returns the corresponding value estimates.

In [19]:
centralised_critic = True
shared_parameters = True

module = MultiAgentMLP(
            n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
            n_agent_outputs=env.action_spec.space.n,
            n_agents=env.n_agents,
            centralised=centralised_critic,
            share_params=shared_parameters,
            device=device,
            depth=2,
            num_cells=256,
            activation_class=nn.Tanh,
        )

In [20]:
value_module = ValueOperator(
            module=module,
            in_keys=[("agents", "observation")],
            out_keys=[("agents", "action_value")],
        )

#### Collector

Collectors perform the following operations:

1. **Reset Environment**: Initialize the environment.
2. **Compute Action**: Determine the next action using the policy and the latest observation.
3. **Execute Step**: Step through the environment with the computed action.

These operations repeat until the environment signals to stop.

In [21]:
collector = SyncDataCollector(
    env,
    policy,
    device=device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
) 

#### Replay buffer

In [22]:
replay_buffer = TensorDictReplayBuffer(
        storage=LazyTensorStorage(memory_size, device=device),
        sampler=SamplerWithoutReplacement(),
        batch_size=minibatch_size,
    )

#### SAC loss function

In [23]:
loss_module = DiscreteSACLoss(
            actor_network=policy,
            qvalue_network=value_module,
            delay_qvalue=True,
            num_actions=env.action_spec.space.n,
            action_space=env.action_spec,
        )

loss_module.set_keys(
    action_value=("agents", "action_value"),
    action=env.action_key,
    reward=env.reward_key,
    done=("agents", "done"),
    terminated=("agents", "terminated"),
)


loss_module.make_value_estimator(ValueEstimators.TD0, gamma=gamma)
target_net_updater = SoftUpdate(loss_module, eps=1 - tau)

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [24]:
total_time = 0
total_frames = 0
sampling_start = time.time()
for i, tensordict_data in enumerate(collector):
    torchrl_logger.info(f"\nIteration {i}")

    sampling_time = time.time() - sampling_start

    current_frames = tensordict_data.numel()
    total_frames += current_frames
    data_view = tensordict_data.reshape(-1)
    replay_buffer.extend(data_view)

    training_tds = []
    training_start = time.time()
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)
            training_tds.append(loss_vals.detach())

            loss_value = (
                loss_vals["loss_actor"]
                + loss_vals["loss_alpha"]
                + loss_vals["loss_qvalue"]
            )

            loss_value.backward()

            total_norm = torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), max_grad_norm
            )
            training_tds[-1].set("grad_norm", total_norm.mean())

            optim.step()
            optim.zero_grad()
            target_net_updater.step()

    collector.update_policy_weights_()

    training_time = time.time() - training_start

    iteration_time = sampling_time + training_time
    total_time += iteration_time
    training_tds = torch.stack(training_tds)

2024-11-19 16:02:48,989 [torchrl][INFO] 
Iteration 0
2024-11-19 16:03:06,578 [torchrl][INFO] 
Iteration 1
2024-11-19 16:03:24,198 [torchrl][INFO] 
Iteration 2
2024-11-19 16:03:40,670 [torchrl][INFO] 
Iteration 3
2024-11-19 16:03:57,879 [torchrl][INFO] 
Iteration 4
2024-11-19 16:04:14,749 [torchrl][INFO] 
Iteration 5
2024-11-19 16:04:31,722 [torchrl][INFO] 
Iteration 6
2024-11-19 16:04:48,887 [torchrl][INFO] 
Iteration 7
2024-11-19 16:05:06,673 [torchrl][INFO] 
Iteration 8
2024-11-19 16:05:23,290 [torchrl][INFO] 
Iteration 9


>  Check `\plots` directory to find the plots created from this experiment.

In [25]:
from RouteRL.services import plotter
plotter(params[kc.PLOTTER])